## 학습 마친 모델 실전 투입

- 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)  

Mounted at /gdrive


- 의존성 패키지 설치

In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.8 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a3d434e835118c28a4dcac08e3c517ffdcd70fe4f5b3c6b83290b5d260b98b6a
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

### 인퍼런스 설정

In [12]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/MyDrive/nlpbook/checkpoint-ner1",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/MyDrive/nlpbook/checkpoint-ner1/epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: /gdrive/MyDrive/nlpbook/checkpoint-ner1/label_map.txt


### 모델 로딩

In [13]:
#체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
#BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
#BERT 모델 초기화
from transformers import BertForTokenClassification
model = BertForTokenClassification(pretrained_model_config)
#체크포인트 주입하기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
#평가 모드로 전환
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [14]:
#토크나이저 초기화
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

### 레이블 맵 작성
범주 인덱스를 범주명과 매칭하는 사전을 만들기

- 문자: (선택 사항) 주어진 문자는 원래 문자열의 시작 또는 끝에서 제거
- 문자 매개변수가 제공되지 않으면 문자열의 시작과 끝에서 공백이 제거

In [15]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

In [16]:
id_to_label

{0: '[CLS]',
 1: '[SEP]',
 2: '[PAD]',
 3: '[MASK]',
 4: 'O',
 5: '인명',
 6: '기타 수량표현',
 7: '기타',
 8: '기관명',
 9: '날짜',
 10: '지명',
 11: '통화',
 12: '비율',
 13: '시간',
 14: '기간',
 15: '인명',
 16: '기타 수량표현',
 17: '기타',
 18: '기관명',
 19: '날짜',
 20: '지명',
 21: '통화',
 22: '비율',
 23: '시간',
 24: '기간'}

### 인퍼런스 함수 선언

In [17]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,   #문장을 토큰화하고 인덱싱하되 max_seq_length보다 짧으면 이에 맞게 패딩하고, 길면 자르기
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})   #inputs를 파이토치 텐서로 변환 후 모델 계산하기
        probs = outputs.logits[0].softmax(dim=1)  #로짓에 소프트맥스를 취해 각 토큰이 어떤 개체명에 속하는지 확률 구하기
        top_probs, preds = torch.topk(probs, dim=1, k=1)  #각 토큰이 속하는 개체명 확률 분포(probs) 가운데 가장 높은 확률값(top_probs)과 그에 속하는 개체명 인덱스(preds) 구하기
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])  #토큰 인덱스 시퀀스(List[int])를 토큰 시퀀스(List[str])로 변환하기
        predicted_tags = [id_to_label[pred.item()] for pred in preds]   #개체명 인덱스 시퀀스(List[int])를 개체명 시퀀스(List[str])로 변환하기
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)   #[CLS], [SEP], [PAD]를 제외한 토큰 각각에 대해 모델이 예측한 개체명(predicted_tag)과 그 확률값(top_probs)을 반환하기
    return {
        "sentence": sentence,
        "result": result,
    }

In [18]:
sentence = "대한민국 인구수는 5천만명이다."
inference_fn(sentence)

{'sentence': '대한민국 인구수는 5천만명이다.',
 'result': [{'token': '대한민국', 'predicted_tag': '기관명', 'top_prob': '0.7882'},
  {'token': '인구', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '##수는', 'predicted_tag': 'O', 'top_prob': '0.9952'},
  {'token': '5천만', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9819'},
  {'token': '##명이', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9898'},
  {'token': '##다', 'predicted_tag': 'O', 'top_prob': '0.9972'},
  {'token': '.', 'predicted_tag': 'O', 'top_prob': '0.9997'}]}

In [19]:
sentence = "한국의 수도는 서울이고 인구는 1천만명이다."
inference_fn(sentence)

{'sentence': '한국의 수도는 서울이고 인구는 1천만명이다.',
 'result': [{'token': '한국의', 'predicted_tag': '지명', 'top_prob': '0.7232'},
  {'token': '수도', 'predicted_tag': 'O', 'top_prob': '0.999'},
  {'token': '##는', 'predicted_tag': 'O', 'top_prob': '0.9992'},
  {'token': '서울', 'predicted_tag': '지명', 'top_prob': '0.9879'},
  {'token': '##이고', 'predicted_tag': 'O', 'top_prob': '0.9992'},
  {'token': '인구', 'predicted_tag': 'O', 'top_prob': '0.9986'},
  {'token': '##는', 'predicted_tag': 'O', 'top_prob': '0.9991'},
  {'token': '1', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9801'},
  {'token': '##천만', 'predicted_tag': '기타 수량표현', 'top_prob': '0.988'},
  {'token': '##명이', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9867'},
  {'token': '##다', 'predicted_tag': 'O', 'top_prob': '0.9982'},
  {'token': '.', 'predicted_tag': 'O', 'top_prob': '0.9998'}]}

### 웹서비스

In [23]:
!mkdir /root/.ngrok2 && echo "authtoken: 2K7HpOJzbZrOEtWlKir6NjCmzOj_7oQuKdBKnQ1fkwDuXUW8o" > /root/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2’: File exists


In [24]:
!ls /root -al

total 64
drwx------ 1 root root 4096 Jan 11 03:17 .
drwxr-xr-x 1 root root 4096 Jan 11 03:09 ..
-r-xr-xr-x 1 root root 1169 Jan  1  2000 .bashrc
drwxr-xr-x 1 root root 4096 Jan 11 03:11 .cache
drwx------ 1 root root 4096 Jan 11 03:09 .config
drwxr-xr-x 5 root root 4096 Jan  9 14:51 .ipython
drwx------ 2 root root 4096 Jan  9 14:51 .jupyter
drwxr-xr-x 2 root root 4096 Jan  9 14:48 .keras
drwxr-xr-x 1 root root 4096 Jan  9 14:51 .local
drwxr-xr-x 2 root root 4096 Jan 11 03:17 .ngrok2
drwxr-xr-x 4 root root 4096 Jan  9 14:51 .npm
-rw-r--r-- 1 root root  148 Aug 17  2015 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  165 Jan  9 14:51 .wget-hsts


In [25]:
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.ner.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9df6-35-196-115-46.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Jan/2023 03:18:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2023 03:18:35] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2023 03:18:52] "POST /api HTTP/1.1" 200 -
